# Big Data Management – Project 3

## Flight Interconnected Data Analysis using Spark and GraphFrames

### Group no: *group 17*  

In [2]:
!pip install graphframes==0.6

  Using cached graphframes-0.6-py2.py3-none-any.whl.metadata (934 bytes)
  Using cached nose-1.3.7-py3-none-any.whl.metadata (1.7 kB)
Using cached graphframes-0.6-py2.py3-none-any.whl (18 kB)
Using cached nose-1.3.7-py3-none-any.whl (154 kB)


In [3]:
import pyspark
from delta import configure_spark_with_delta_pip
from pyspark.sql import functions as F
from pyspark.sql import Window
from graphframes import GraphFrame
import random

In [4]:
builder = pyspark.sql.SparkSession.builder.appName("FlightGraphAnalysis") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200")

spark = configure_spark_with_delta_pip(builder,extra_packages=["graphframes:graphframes:0.8.4-spark3.5-s_2.12"]).getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", spark._sc.defaultParallelism)

spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # OK for exploration, not great for performance
spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)

## Graph Construction

We load flight data from the 2009 dataset and construct a graph where:
- **Vertices** represent airports (based on `ORIGIN` and `DEST` fields).
- **Edges** represent directed flights from one airport to another.

The graph is constructed using GraphFrames for further analysis.

In [5]:
airport_df = spark.read.csv("2009.csv", header=True, inferSchema=True)

In [6]:
display(airport_df)

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202,1206.0,4.0,0.0,NULL,0.0,62.0,68.0,42.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632,1624.0,-8.0,0.0,NULL,0.0,82.0,75.0,43.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210,1201.0,-9.0,0.0,NULL,0.0,70.0,62.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357,1345.0,-12.0,0.0,NULL,0.0,77.0,56.0,37.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900,1822.0,-38.0,0.0,NULL,0.0,105.0,77.0,40.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,ATL,EWR,1915,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142,2123.0,-19.0,0.0,NULL,0.0,147.0,130.0,96.0,745.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,CLE,ATL,1645,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842,1825.0,-17.0,0.0,NULL,0.0,117.0,108.0,91.0,554.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1214,DCA,EWR,1915,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035,2027.0,-8.0,0.0,NULL,0.0,80.0,79.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1215,EWR,DCA,1715,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838,1823.0,-15.0,0.0,NULL,0.0,83.0,73.0,41.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1217,EWR,DCA,1300,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408,1356.0,-12.0,0.0,NULL,0.0,68.0,61.0,39.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL


In [7]:
vertices = airport_df.select(F.col("ORIGIN").alias("id")) \
    .union(airport_df.select(F.col("DEST").alias("id"))) \
    .distinct() \
    .repartition(200) \
    .cache()

edges = airport_df.select(
    F.col("ORIGIN").alias("src"),
    F.col("DEST").alias("dst")
).repartition(200).cache()

In [8]:
from graphframes import GraphFrame

# Defining GraphFrame from existing vertices and edges
g = GraphFrame(vertices, edges)

/usr/local/spark/python/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


### Graph Basic Statistics

This table shows the number of vertices (airports) and edges (flights) in the graph. It provides an overview of the dataset's structure after conversion to a graph format.

In [9]:
print("\n=== Graph Basic Statistics ===")
counts_data = [("Vertices (airports)", vertices.count()), 
               ("Edges (flights)", edges.count())]

counts_df = spark.createDataFrame(counts_data, ["Component", "Count"])
display(counts_df)


=== Graph Basic Statistics ===


Component,Count
Vertices (airports),296
Edges (flights),6429338


## Query 1: Degree Statistics

We compute:
- **In-Degree**: Number of flights arriving at each airport.
- **Out-Degree**: Number of flights departing from each airport.
- **Total Degree**: Sum of in- and out-degrees, showing overall connectivity.
- **Triangle count**: The number of triangles each airport is part of. A triangle represents a tightly interconnected group of three airports with direct flights among them.

This helps identify highly connected airports.

In [10]:
print("\n=== Query 1: Degree Statistics ===")

from pyspark.sql.functions import least, greatest, collect_set, array, explode, col, lit, sum as spark_sum

# Degree Stats
degree_stats = edges.select(
    col("src").alias("node"),
    lit(1).alias("out_degree"),
    lit(0).alias("in_degree")
).union(
    edges.select(
        col("dst").alias("node"),
        lit(0).alias("out_degree"),
        lit(1).alias("in_degree")
    )
).groupBy("node").agg(
    spark_sum("out_degree").alias("out_degree"),
    spark_sum("in_degree").alias("in_degree")
).withColumn(
    "total_degree", col("out_degree") + col("in_degree")
)

# Triangle Count per Node (custom implementation)
def custom_triangle_count_per_node(g):
    undirected_edges = g.edges.select(
        least("src", "dst").alias("a"),
        greatest("src", "dst").alias("b")
    ).distinct()

    neighbors = undirected_edges.groupBy("a").agg(
        collect_set("b").alias("neighbors")
    )

    triangles = undirected_edges.alias("edges").join(
        neighbors.alias("neigh"),
        col("edges.a") == col("neigh.a"),
        "inner"
    ).selectExpr(
        "edges.a",
        "edges.b",
        "explode(neigh.neighbors) as c"
    ).filter(
        col("b") < col("c")
    )

    vertex_triangle_counts = triangles.select(
        explode(array("a", "b", "c")).alias("id")
    ).groupBy("id").count()

    return vertex_triangle_counts.withColumnRenamed("count", "triangle_count")

# Join triangle counts to degree stats
triangle_counts = custom_triangle_count_per_node(g).withColumnRenamed("id", "node")

full_stats = degree_stats.join(triangle_counts, on="node", how="left").fillna(0)

# Show result
full_stats.orderBy(col("total_degree").desc()).show()



=== Query 1: Degree Statistics ===
+----+----------+---------+------------+--------------+
|node|out_degree|in_degree|total_degree|triangle_count|
+----+----------+---------+------------+--------------+
| ATL|    417449|   417457|      834906|         12183|
| ORD|    313848|   313769|      627617|          2687|
| DFW|    264396|   264398|      528794|          5995|
| DEN|    235675|   235700|      471375|          4749|
| LAX|    192879|   192916|      385795|          2166|
| PHX|    183502|   183491|      366993|          1934|
| IAH|    182097|   182088|      364185|          2923|
| LAS|    153993|   153984|      307977|          2213|
| DTW|    152081|   152075|      304156|          4692|
| SFO|    136488|   136532|      273020|          1575|
| SLC|    131694|   131674|      263368|          1663|
| MCO|    120944|   120936|      241880|          1931|
| MSP|    119732|   119759|      239491|          2401|
| JFK|    119574|   119571|      239145|          1801|
| EWR|    11

## Query 2: Triangle Count (Custom Implementation)

This cell uses a set-join approach to identify all unique triangles in the undirected airport graph.
It ensures that each triangle (a, b, c) is only counted once by normalizing edge directions using `least()` and `greatest()`.

The pattern matched is: (a-b), (b-c), and (a-c).

In [11]:
print("\n=== Query 2: Triangle Count (Custom Implementation) ===")
from pyspark.sql.functions import least, greatest, col

# Convert directed edges to undirected (a, b) where a < b
undirected_edges = edges.withColumn("u", least("src", "dst")).withColumn("v", greatest("src", "dst")) \
                        .select("u", "v").distinct()

# Rename for triangle finding
e1 = undirected_edges.select(col("u").alias("a"), col("v").alias("b"))
e2 = undirected_edges.select(col("u").alias("b"), col("v").alias("c"))
e3 = undirected_edges.select(col("u").alias("a"), col("v").alias("c"))

# Join on undirected edge pattern: a-b, b-c, a-c
triangles = e1.join(e2, "b").join(e3, ["a", "c"])

# Each triangle appears 6 times due to symmetry (a-b-c, b-c-a, c-a-b in both directions)
triangle_count = triangles.select("a", "b", "c").distinct().count() 

print(f"Total triangles in the undirected graph: {triangle_count}")


=== Query 2: Triangle Count (Custom Implementation) ===
Total triangles in the undirected graph: 16015


### Triangle count using graphframes functions (used only for comparison)

In [12]:
def exact_triangle_count_with_graphframes(edges, vertices=None):
    """
    Count exact number of triangles using GraphFrames
    
    Args:
        edges: DataFrame with 'src' and 'dst' columns
        vertices: Optional DataFrame with vertex IDs (if None, will be derived from edges)
    
    Returns:
        Exact count of triangles in the graph
    """
  
    # If vertices are not provided, extract them from edges
    if vertices is None:
        # Extract all unique vertices from edges
        vertices_from_src = edges.select(F.col("src").alias("id")).distinct()
        vertices_from_dst = edges.select(F.col("dst").alias("id")).distinct()
        vertices = vertices_from_src.union(vertices_from_dst).distinct()
    
    # Ensure edges are distinct (no duplicate edges)
    distinct_edges = edges.distinct()
    
    # Create a GraphFrame
    g = GraphFrame(vertices, distinct_edges)
    
    # Find triangles
    triangles = g.triangleCount()
    
    # Sum up the triangle counts
    # The count() method gives us the total number of triangles
    total_triangles = triangles.groupBy().agg(F.sum("count").alias("total")).collect()[0]["total"]
    
    # GraphFrames counts each triangle once for each vertex, so we need to divide by 3
    return int(total_triangles / 3)

In [13]:
exact_triangle_count = exact_triangle_count_with_graphframes(edges)
print(f"Exact triangle count: {exact_triangle_count}")

/usr/local/spark/python/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/spark/python/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


Exact triangle count: 16015


## Query 3: Centrality (Degree Centrality)

We calculate **degree centrality** for each airport using the formula:

degree_centrality = total_degree / (number of nodes - 1)

This metric identifies how "influential" an airport is based on its number of direct connections.


In [14]:
print("\n=== Query 3: Degree Centrality ===")

centrality = degree_stats.select(
    "node",
    (F.col("total_degree") / F.lit(vertices.count() - 1)).alias("degree_centrality")
).orderBy(F.col("degree_centrality").desc())

display(centrality.limit(10))


=== Query 3: Degree Centrality ===


node,degree_centrality
ATL,2830.1898305084746
ORD,2127.515254237288
DFW,1792.5220338983052
DEN,1597.8813559322034
LAX,1307.7796610169491
PHX,1244.04406779661
IAH,1234.5254237288136
LAS,1043.9898305084746
DTW,1031.0372881355931
SFO,925.4915254237288


## Query 4: PageRank (Custom Implementation)

Here we manually implement the **PageRank algorithm**:
- Each airport starts with an equal rank.
- At each iteration, rank is redistributed from source to destination nodes.
- A damping factor (commonly 0.85) is applied to simulate teleportation probability.

We repeat this for 10 iterations and rank airports by their final scores.


In [15]:
print("\n=== Query 4: Custom PageRank ===")

num_iterations = 10
damping_factor = 0.85
total_nodes = vertices.count()

# Initialize ranks
ranks = vertices.select("id").withColumn("rank", F.lit(1.0 / total_nodes)).cache()

for i in range(num_iterations):
    contributions = edges.join(ranks, edges.src == ranks.id) \
        .groupBy("dst").agg(F.sum("rank").alias("total_contrib"))
    
    new_ranks = vertices.join(contributions, vertices.id == contributions.dst, "left") \
        .fillna(0, subset=["total_contrib"]) \
        .select(
            "id",
            ((damping_factor * F.col("total_contrib")) + 
            ((1 - damping_factor) / total_nodes)).alias("rank")
        ).cache()
    
    ranks.unpersist()
    ranks = new_ranks

pagerank_results = ranks.orderBy(F.col("rank").desc())
# display(pagerank_results.limit(10))
display(pagerank_results)


=== Query 4: Custom PageRank ===


id,rank
ATL,2.3561081790300507E47
ORD,2.1475525152182944E47
DFW,1.8337063895243112E47
DEN,1.7879699261447383E47
LAX,1.7327029344334827E47
PHX,1.5514294916968752E47
LAS,1.4656286648064942E47
IAH,1.3025335044602526E47
SFO,1.2498321865847688E47
BOS,1.1807420077410707E47


## Query 5: Most Connected Airports

We sort airports by their **total degree** to identify the most connected hubs in the network — those with the highest number of incoming and outgoing flights combined.

In [16]:
print("\n=== Query 5: Most Connected Airports ===")

most_connected = degree_stats.orderBy(F.col("total_degree").desc()).limit(10)
display(most_connected)


=== Query 5: Most Connected Airports ===


node,out_degree,in_degree,total_degree
ATL,417449,417457,834906
ORD,313848,313769,627617
DFW,264396,264398,528794
DEN,235675,235700,471375
LAX,192879,192916,385795
PHX,183502,183491,366993
IAH,182097,182088,364185
LAS,153993,153984,307977
DTW,152081,152075,304156
SFO,136488,136532,273020
